In [109]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [2]:
train = pd.read_csv("Australian.csv")
train.head()

,1,22.08,11.46,2,4,4.1,1.585,0,0.1,0.2,1.1,2.1,100,1213,0.3
0,0,22.67,7.000,2,8,4,0.165,0,0,0,0,2,160,1,0
1,0,29.58,1.750,1,4,4,1.250,0,0,0,1,2,280,1,0
2,0,21.67,11.500,1,5,3,0.000,1,1,11,1,2,0,1,1
3,1,20.17,8.170,2,6,4,1.960,1,1,14,0,2,60,159,1
4,0,15.83,0.585,2,8,8,1.500,1,1,2,0,2,100,1,1


In [3]:
train['0.3'] = train['0.3'].map({1: 1, 0: -1})
print(train['0.3'])

0     -1
1     -1
2      1
3      1
4      1
5     -1
6      1
7     -1
8     -1
9      1
10     1
11    -1
12     1
13    -1
14     1
15     1
16     1
17    -1
18     1
19    -1
20    -1
21    -1
22    -1
23    -1
24     1
25    -1
26     1
27     1
28     1
29     1
      ..
659    1
660   -1
661    1
662    1
663   -1
664   -1
665   -1
666   -1
667   -1
668   -1
669    1
670    1
671   -1
672    1
673    1
674   -1
675   -1
676   -1
677   -1
678    1
679   -1
680   -1
681    1
682   -1
683    1
684    1
685   -1
686    1
687    1
688    1
Name: 0.3, Length: 689, dtype: int64


In [4]:
features = train.columns[0:14]
X = train[features]
y = train['0.3']
train.head()

,1,22.08,11.46,2,4,4.1,1.585,0,0.1,0.2,1.1,2.1,100,1213,0.3
0,0,22.67,7.000,2,8,4,0.165,0,0,0,0,2,160,1,-1
1,0,29.58,1.750,1,4,4,1.250,0,0,0,1,2,280,1,-1
2,0,21.67,11.500,1,5,3,0.000,1,1,11,1,2,0,1,1
3,1,20.17,8.170,2,6,4,1.960,1,1,14,0,2,60,159,1
4,0,15.83,0.585,2,8,8,1.500,1,1,2,0,2,100,1,1


In [41]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2,random_state=0)


In [42]:
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)

In [43]:
print(X_train)

[[0.000e+00 4.600e+01 4.000e+00 ... 2.000e+00 1.000e+02 9.610e+02]
 [1.000e+00 2.250e+01 1.150e+01 ... 2.000e+00 0.000e+00 4.001e+03]
 [1.000e+00 4.117e+01 1.335e+00 ... 2.000e+00 1.680e+02 1.000e+00]
 ...
 [1.000e+00 3.200e+01 1.750e+00 ... 2.000e+00 3.930e+02 1.000e+00]
 [0.000e+00 3.325e+01 2.500e+00 ... 2.000e+00 0.000e+00 3.000e+00]
 [1.000e+00 3.157e+01 1.050e+01 ... 2.000e+00 0.000e+00 1.000e+00]]


In [65]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

In [121]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [122]:

if __name__ == "__main__":
    import pylab as pl   
    def test_non_linear():
        
        clf = SVM(C=100.0)
        clf.fit(X_train, y_train)

        y_predict = clf.predict(X_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print(current/len(y_predict))
        
    def test_soft():

        clf = SVM(C=100.0)
        clf.fit(X_train, y_train)

        y_predict = clf.predict(X_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print(correct/len(y_predict))

    test_soft()
    #test_non_linear()
    

(551, 551)
     pcost       dcost       gap    pres   dres
 0:  2.8131e+05 -1.4070e+06  2e+06  1e-12  3e-14
 1:  8.3050e+04 -1.7372e+05  3e+05  5e-13  1e-14
 2:  1.5026e+04 -3.4021e+04  5e+04  6e-14  7e-15
 3:  1.6031e+03 -4.7148e+03  6e+03  2e-13  4e-15
 4: -1.7821e+02 -9.6138e+02  8e+02  6e-14  1e-15
 5: -3.2787e+02 -4.8242e+02  2e+02  9e-15  6e-16
 6: -3.4204e+02 -3.5546e+02  1e+01  3e-14  6e-16
 7: -3.4254e+02 -3.4408e+02  2e+00  4e-15  6e-16
 8: -3.4261e+02 -3.4269e+02  9e-02  2e-14  6e-16
 9: -3.4261e+02 -3.4262e+02  8e-03  1e-14  5e-16
10: -3.4261e+02 -3.4261e+02  2e-04  3e-14  6e-16
Optimal solution found.
optimal
542 support vectors out of 551 points
87 out of 138 predictions correct
0.6304347826086957
